In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

int(config['augmentations']['sample_rate'])

In [ ]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import machineLearning
from AudioDataset import AudioDataset
import Augmentation
import random
import seaborn as sn
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_paths = [str(p) for p in Path('./saved_model/').glob(f'*.pt')]
for i, model_path in enumerate(model_paths):
    print(f'[{i}] {model_path}')

path = model_paths[int(input('Select saved model > '))]
model = torch.load(path, map_location=device)


In [ ]:
import torch
import torchaudio
from torchaudio.io import StreamReader
from Augmentation import Augmentor

augmentor = Augmentor()

streamer = StreamReader(
    src="audio=@device_cm_{33D9A762-90C8-11D0-BD43-00A0C911CE86}\wave_{07DEF3C3-A487-4CE6-A6E3-535301DF2D46}",
    format="dshow",
)

streamer.add_basic_audio_stream(
    frames_per_chunk=44100*4, sample_rate=44100)

class_map = ['air conditioner', 'car horn', 'children playing', 'dog bark',
             'drilling', 'engine idling', 'gunshot', 'jackhammer', 'siren', 'street music']
             
print(streamer.get_src_stream_info(0))
print(streamer.get_out_stream_info(0))
stream_iterator = streamer.stream()
wav = torch.Tensor([])
sm = torch.nn.Softmax()
model.eval()
with torch.no_grad():
    while True:
        (chunk,) = next(stream_iterator)
        # wav = torch.cat((wav, chunk[:, 0]))
        spectrogram = torchaudio.transforms.Spectrogram()
        wav, sr = augmentor.audio_preprocessing([chunk.T, 44100])
        
        spectrogram_tensor = (spectrogram(wav) + 1e-12).log2()
        pred = model(torch.unsqueeze(spectrogram_tensor, 0))
        pred = sm(pred)
        print(class_map[pred.argmax()])
        sn.barplot(y=class_map, x=pred[0].cpu().numpy())
        plt.show()

streamer.remove_stream(0)


In [ ]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()


print(frames)


In [ ]:
import torch
from pathlib import Path
import testModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_paths = [str(p) for p in Path('./saved_model/').glob(f'*.pt')]
for i, model_path in enumerate(model_paths):
    print(f'[{i}] {model_path}')

path = model_paths[int(input('Select saved model > '))]
model = torch.load(path, map_location=device)
testModel.predictFile('./test/test.wav', model, device)
testModel.predictFolder('./data/testset/', model, device)


In [ ]:
pred.cpu()